In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

from glob import glob
import tensorflow as tf
import keras
from keras import layers


"""
## Define the Transformer Input Layer

When processing past target tokens for the decoder, we compute the sum of
position embeddings and token embeddings.

When processing audio features, we apply convolutional layers to downsample
them (via convolution strides) and process local relationships.
"""


class TokenEmbedding(layers.Layer):
    def __init__(self, num_vocab=1000, maxlen=100, num_hid=64):
        super().__init__()
        self.emb = keras.layers.Embedding(num_vocab, num_hid)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        x = self.emb(x)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions


class SpeechFeatureEmbedding(layers.Layer):
    def __init__(self, num_hid=64, maxlen=100):
        super().__init__()
        self.conv1 = keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv2 = keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv3 = keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return self.conv3(x)


"""
## Transformer Encoder Layer
"""


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


"""
## Transformer Decoder Layer
"""


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.1):
        super().__init__()
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.self_att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.enc_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.self_dropout = layers.Dropout(0.5)
        self.enc_dropout = layers.Dropout(0.1)
        self.ffn_dropout = layers.Dropout(0.1)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        """Masks the upper half of the dot product matrix in self attention.

        This prevents flow of information from future tokens to current token.
        1's in the lower triangle, counting from the lower right corner.
        """
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)

    def call(self, enc_out, target):
        input_shape = tf.shape(target)
#         print("input_shape",input_shape)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        target_att = self.self_att(target, target, attention_mask=causal_mask)
        target_norm = self.layernorm1(target + self.self_dropout(target_att))
        enc_out = self.enc_att(target_norm, enc_out)
        enc_out_norm = self.layernorm2(self.enc_dropout(enc_out) + target_norm)
        ffn_out = self.ffn(enc_out_norm)
        ffn_out_norm = self.layernorm3(enc_out_norm + self.ffn_dropout(ffn_out))
        return ffn_out_norm


"""
## Complete the Transformer model

Our model takes audio spectrograms as inputs and predicts a sequence of characters.
During training, we give the decoder the target character sequence shifted to the left
as input. During inference, the decoder uses its own past predictions to predict the
next token.
"""


class Transformer(keras.Model):
    def __init__(
        self,
        num_hid=64,
        num_head=2,
        num_feed_forward=128,
        source_maxlen=100,
        target_maxlen=100,
        num_layers_enc=4,
        num_layers_dec=1,
        num_classes=10,
        **kwargs
    ):
        super().__init__(**kwargs)
        
        self.num_hid = num_hid
        self.num_head = num_head
        self.num_feed_forward = num_feed_forward
        self.source_maxlen = source_maxlen
        self.target_maxlen = target_maxlen
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.num_classes = num_classes
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.target_maxlen = target_maxlen
        self.num_classes = num_classes

        self.enc_input = SpeechFeatureEmbedding(num_hid=num_hid, maxlen=source_maxlen)
        self.dec_input = TokenEmbedding(
            num_vocab=num_classes, maxlen=target_maxlen, num_hid=num_hid
        )

        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )

        for i in range(num_layers_dec):
            setattr(
                self,
                f"dec_layer_{i}",
                TransformerDecoder(num_hid, num_head, num_feed_forward),
            )

        self.classifier = layers.Dense(num_classes)

    def decode(self, enc_out, target):
        y = self.dec_input(target)
        for i in range(self.num_layers_dec):
            y = getattr(self, f"dec_layer_{i}")(enc_out, y)
        return y

    def call(self, inputs):
        source = inputs[0]
        target = inputs[1]
        x = self.encoder(source)
        y = self.decode(x, target)
        return self.classifier(y)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "num_hid": self.num_hid,
            "num_head": self.num_head,
            "num_feed_forward": self.num_feed_forward,
            "source_maxlen": self.source_maxlen,
            "target_maxlen": self.target_maxlen,
            "num_layers_enc": self.num_layers_enc,
            "num_layers_dec": self.num_layers_dec,
            "num_classes": self.num_classes,
        })
        return config

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch):
        """Processes one batch inside model.fit()."""
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        with tf.GradientTape() as tape:
            preds = self([source, dec_input])
            one_hot = tf.one_hot(dec_target, depth=self.num_classes)
            mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
            loss = model.compute_loss(None, one_hot, preds, sample_weight=mask)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def test_step(self, batch):
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        preds = self([source, dec_input])
        one_hot = tf.one_hot(dec_target, depth=self.num_classes)
        mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
        loss = model.compute_loss(None, one_hot, preds, sample_weight=mask)
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def generate(self, source, target_start_token_idx):
        """Performs inference over one batch of inputs using greedy decoding."""
        bs = tf.shape(source)[0]
        enc = self.encoder(source)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        dec_logits = []
        for i in range(self.target_maxlen - 1):
            dec_out = self.decode(enc, dec_input)
            logits = self.classifier(dec_out)
            logits = tf.argmax(logits, axis=-1, output_type=tf.int32)
            last_logit = tf.expand_dims(logits[:, -1], axis=-1)
            dec_logits.append(last_logit)
            dec_input = tf.concat([dec_input, last_logit], axis=-1)
        return dec_input





2024-07-01 04:29:06.091940: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 04:29:06.092049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 04:29:06.209693: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [22]:
import os 
from glob import glob
from numpy import vectorize 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch




import os
import pandas as pd

def get_data(csv_path1, csv_path2, lim1=32000, lim2=2500):
    """
    Reads two CSV files, preprocesses data, and extracts audio and transcript information.

    Args:
      csv_path1: Path to the first CSV file containing audio and text data.
      csv_path2: Path to the second CSV file containing audio and text data.
      lim1: Limit on the number of rows to read from the first CSV file (default: 50000).
      lim2: Limit on the number of rows to read from the second CSV file (default: 2500).

    Returns:
      A list of dictionaries containing "audio" and "text" keys.
    """
    # Read and process the first CSV
    df1 = pd.read_csv(csv_path1, encoding='utf-8')
    df1 = df1.head(lim1)  # Read only the first 'lim1' rows
    df1 = df1[df1['audio'].notna() & df1['transcript'].notna()]
    df1 = df1[~df1['transcript'].str.contains(r'\s<\s|\[|÷|=', na=False)]
    df1 = df1.rename(columns={"audio": "ID"})
    df1['file_path'] = df1['ID'].apply(lambda x: os.path.join(train_files_path, x + ".wav"))

    # Read and process the second CSV
    df2 = pd.read_csv(csv_path2, encoding='utf-8')
    df2 = df2.head(lim2)  # Read only the first 'lim2' rows
    df2 = df2[df2['audio'].notna() & df2['transcript'].notna()]
    df2 = df2[~df2['transcript'].str.contains(r'\s<\s|\[|÷|=', na=False)]
    df2 = df2.rename(columns={"audio": "ID"})
    df2['file_path'] = df2['ID'].apply(lambda x: os.path.join(adapt_files_path, x + ".wav"))

    # Concatenate the DataFrames
    df = pd.concat([df1, df2], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True)

    # Create the final data list
    data = df[["file_path", "transcript"]].to_dict(orient='records')
    data = [{'audio': item['file_path'], 'text': item['transcript']} for item in data]

    return data


"""
## Preprocess the dataset
"""


class VectorizeChar:
    def __init__(self, max_len=50):
        self.vocab = (
                        ["-", "#", "<", ">"]
                        + ['آ', 'إ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ'
                            , 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ',
                             'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي', 'أ', 'ؤ'
                             , 'ئ', 'ة', 'ى', 'ء', ' ', 'f','o','l', "،", "ٱ","ڨ", "چ"]
                        + [" ", ".", ",", "?"]
        )
        self.max_len = max_len
        self.char_to_idx = {}
        for i, ch in enumerate(self.vocab):
            self.char_to_idx[ch] = i

    def __call__(self, text):
        text = text.lower()
        text = text[: self.max_len - 2]
        text = "<" + text + ">"
        pad_len = self.max_len - len(text)
        return [self.char_to_idx.get(ch, 1) for ch in text] + [0] * pad_len

    def get_vocabulary(self):
        return self.vocab

train_csv = "/kaggle/input/mtc-asr/train.csv"
adapt_csv = "/kaggle/input/mtc-asr/adapt.csv"

train_files_path = "/kaggle/input/mtc-asr/train"
adapt_files_path = "/kaggle/input/mtc-asr/adapt"
output_path = "/kaggle/working"

max_target_len = 200  # all transcripts in out data are < 200 characters
data = get_data(train_csv, adapt_csv)
vectorizer = VectorizeChar(max_target_len)
print("vocab size", len(vectorizer.get_vocabulary()))


def create_text_ds(data):
    texts = [_["text"] for _ in data]
    text_ds = [vectorizer(t) for t in texts]
    text_ds = tf.data.Dataset.from_tensor_slices(text_ds)
    return text_ds

def create_id_ds(data):
    texts = [_["audio"] for _ in data]
    text_ds = [vectorizer(t) for t in texts]
    text_ds = tf.data.Dataset.from_tensor_slices(text_ds)
    return text_ds


def path_to_audio(path):
    # spectrogram using stft
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1)
    audio = tf.squeeze(audio, axis=-1)
    stfts = tf.signal.stft(audio, frame_length=200, frame_step=80, fft_length=256)
    x = tf.math.pow(tf.abs(stfts), 0.5)
    mx=tf.math.reduce_max(x)
    # normalisation
    means = tf.math.reduce_mean(x, 1, keepdims=True)
    stddevs = tf.math.reduce_std(x, 1, keepdims=True)
    epsilon = 1e-7  # Very small number
    stddevs = tf.where(stddevs == 0, epsilon, stddevs)  # Replace zero stddevs with epsilon
    
    x = (x - means) / stddevs

    audio_len = tf.shape(x)[0]
    # padding to 10 seconds
    pad_len = int(2754*1.5)
    paddings = tf.constant([[0, pad_len], [0, 0]])
    x = tf.pad(x, paddings, "CONSTANT")[:pad_len, :]
    return x


def create_audio_ds(data):
    flist = [_["audio"] for _ in data]
    audio_ds = tf.data.Dataset.from_tensor_slices(flist)
    audio_ds = audio_ds.map(path_to_audio, num_parallel_calls=tf.data.AUTOTUNE)
    return audio_ds


def create_tf_dataset(data, bs=4):
    audio_ds = create_audio_ds(data)
    text_ds = create_text_ds(data)
    ds = tf.data.Dataset.zip((audio_ds, text_ds))
    ds = ds.map(lambda x, y: {"source": x, "target": y})
    ds = ds.batch(bs)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


split = int(len(data) * 0.99)
train_data = data[:split]
test_data = data[split-20:split]

ds = create_tf_dataset(train_data, bs=64)

val_ds = create_tf_dataset(test_data, bs=4)

"""
### Callbacks to display predictions
"""



vocab size 52


'\n### Callbacks to display predictions\n'

In [15]:

class DisplayOutputs(keras.callbacks.Callback):
    def __init__(
        self, batch, idx_to_token, target_start_token_idx=27, target_end_token_idx=28
    ):
        """Displays a batch of outputs after every epoch

        Args:
            batch: A test batch containing the keys "source" and "target"
            idx_to_token: A List containing the vocabulary tokens corresponding to their indices
            target_start_token_idx: A start token index in the target vocabulary
            target_end_token_idx: An end token index in the target vocabulary
        """
        self.batch = batch
        self.target_start_token_idx = target_start_token_idx
        self.target_end_token_idx = target_end_token_idx
        self.idx_to_char = idx_to_token

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 != 0:
            return
        source = self.batch["source"]
        target = self.batch["target"].numpy()
        bs = tf.shape(source)[0]
        preds = self.model.generate(source, self.target_start_token_idx)
        preds = preds.numpy()
        target_words=[]
        pred=[]
        for i in range(bs-1,bs):
            target_text = "".join([self.idx_to_char[_] for _ in target[i, :]])
            prediction = ""
            for idx in preds[i, :]:
                prediction += self.idx_to_char[idx]
                if idx == self.target_end_token_idx:
                    break
            target_words+=[target_text.replace('-','')]
            pred+=[prediction]
        for i in range(1):
            print(f"target:     {target_words[i]}")
            print(f"prediction: {pred[i]}\n")

    def on_test_batch_end(self, batch, logs=None):
        source = self.batch["source"]
        target = self.batch["target"].numpy()
        bs = tf.shape(source)[0]
        preds = self.model.generate(source, self.target_start_token_idx)
        preds = preds.numpy()
        
            

"""
## Learning rate schedule
"""


class CustomSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(
        self,
        init_lr=0.00001,
        lr_after_warmup=0.001,
        final_lr=0.00001,
        warmup_epochs=15,
        decay_epochs=85,
        steps_per_epoch=203,
    ):
        super().__init__()
        self.init_lr = init_lr
        self.lr_after_warmup = lr_after_warmup
        self.final_lr = final_lr
        self.warmup_epochs = warmup_epochs
        self.decay_epochs = decay_epochs
        self.steps_per_epoch = steps_per_epoch

    def calculate_lr(self, epoch):
        """linear warm up - linear decay"""
        warmup_lr = (
            self.init_lr
            + ((self.lr_after_warmup - self.init_lr) / (self.warmup_epochs - 1)) * epoch
        )
        decay_lr = tf.math.maximum(
            self.final_lr,
            self.lr_after_warmup
            - (epoch - self.warmup_epochs)
            * (self.lr_after_warmup - self.final_lr)
            / self.decay_epochs,
        )
        return tf.math.minimum(warmup_lr, decay_lr)

    def __call__(self, step):
        epoch = step // self.steps_per_epoch
        epoch = tf.cast(epoch, "float32")
        return self.calculate_lr(epoch)
    
    def get_config(self):
        config = {
            "init_lr": self.init_lr,
            "lr_after_warmup": self.lr_after_warmup,
            "final_lr": self.final_lr,
            "warmup_epochs": self.warmup_epochs,
            "decay_epochs": self.decay_epochs,
            "steps_per_epoch": self.steps_per_epoch,
        }
        return config


In [ ]:
# import shutil
# shutil.rmtree("/kaggle/working/ASAR-v2/backup/")

In [14]:
newpath = r'/kaggle/working/ASAR-v2/backup' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [5]:
backup_dir= r'/kaggle/working/ASAR-v2/backup'
train_checkpoint = keras.callbacks.BackupAndRestore(
    backup_dir, save_freq="epoch", delete_checkpoint=False
)

In [17]:

"""
## Create & train the end-to-end model
"""

batch = next(iter(val_ds))

# The vocabulary to convert predicted indices into characters
idx_to_char = vectorizer.get_vocabulary()
display_cb = DisplayOutputs(
    batch, idx_to_char, target_start_token_idx=2, target_end_token_idx=3
)  # set the arguments as per vocabulary index for '<' and '>'


In [20]:
newpath = r'/kaggle/working/ASAR-v3/' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
filepath="/kaggle/working/ASAR-v3/weights-improvement-{epoch:02d}.model.keras"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')


In [23]:

# backup_dir= r'/kaggle/working/ASAR-v2/backup'

# checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')

# train_checkpoint = keras.callbacks.BackupAndRestore(
#     backup_dir, save_freq="epoch", delete_checkpoint=False
# )

# """
# ## Create & train the end-to-end model
# """

# batch = next(iter(val_ds))

# # The vocabulary to convert predicted indices into characters
# idx_to_char = vectorizer.get_vocabulary()
# display_cb = DisplayOutputs(
#     batch, idx_to_char, target_start_token_idx=2, target_end_token_idx=3
# )  # set the arguments as per vocabulary index for '<' and '>'



# model = Transformer(
#     num_hid=200,
#     num_head=2,
#     num_feed_forward=400,
#     target_maxlen=max_target_len,
#     num_layers_enc=4,
#     num_layers_dec=1,
#     num_classes=52,
# )
# loss_fn = keras.losses.CategoricalCrossentropy(
#     from_logits=True,
#     label_smoothing=0.1,
# )

# learning_rate = CustomSchedule(
#     init_lr=0.00001,
#     lr_after_warmup=0.001,
#     final_lr=0.00001,
#     warmup_epochs=15,
#     decay_epochs=85,
#     steps_per_epoch=len(ds),
# )
# optimizer = keras.optimizers.Adam(learning_rate)

# # load_epoch=2
# # if os.path.exists(filepath.format(epoch=load_epoch)):
# #     model.load_model(filepath.format(epoch=load_epoch))
# #     print("Checkpoint loaded.")
# checkpoint_path="/kaggle/working/ASAR-v2/backup/latest.weights.h5"
# # model.load_weights(checkpoint_path)

# model.compile(optimizer=optimizer, loss=loss_fn)
# history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb],epochs=1)

# history = model.fit(ds, validation_data=val_ds, callbacks=[checkpoint,train_checkpoint,display_cb],epochs=100)

# model.save('my_model_v3.keras')

# """
# In practice, you should train for around 100 epochs or more.

# Some of the predicted text at or around epoch 35 may look as follows:

# target:     <as they sat in the car, frazier asked oswald where his lunch was>
# prediction: <as they sat in the car frazier his lunch ware mis lunch was>

# target:     <under the entry for may one, nineteen sixty,>
# prediction: <under the introus for may monee, nin the sixty,>

# """

W0000 00:00:1719810048.423230     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


576/576 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - loss: 1.2121

W0000 00:00:1719810252.273900     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
/opt/conda/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (4, 2, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


target:     <دفع ضريبة الحريه الدم الدم واسترداد الأرض والعزه والكرامه ليس لها خيار اخر>
prediction: <و ال ال ال ال ال الال ال الي الي ال ال الي ال الي ال الي الالي ال ا الا الي ا الي ال الي الالي ال بي الي ال مالي م الي الي م ال اللم ال ا ال ا ا ال الم الي الي م الي ا و ا الي الماللي ال ا عل المالي 

576/576 ━━━━━━━━━━━━━━━━━━━━ 268s 422ms/step - loss: 1.2119 - val_loss: 0.9292
Epoch 1/100
576/576 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - loss: 0.9978
Epoch 1: saving model to /kaggle/working/ASAR-v3/weights-improvement-01.model.keras
target:     <دفع ضريبة الحريه الدم الدم واسترداد الأرض والعزه والكرامه ليس لها خيار اخر>
prediction: <و اللالالي الل الي الي الي الي اللي اللي الي الي الي الي الي اللي الي الي الي الي الي الي الي الليه اليه الي الي الي الي الي اليه اللمالي الللليه مالولي مالي اللي الي الي المالللماللي اللال الالمالي>

576/576 ━━━━━━━━━━━━━━━━━━━━ 229s 397ms/step - loss: 0.9978 - val_loss: 0.8864
Epoch 2/100
576/576 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - loss: 0.9732
Epoch 2: sav

'\nIn practice, you should train for around 100 epochs or more.\n\nSome of the predicted text at or around epoch 35 may look as follows:\n\ntarget:     <as they sat in the car, frazier asked oswald where his lunch was>\nprediction: <as they sat in the car frazier his lunch ware mis lunch was>\n\ntarget:     <under the entry for may one, nineteen sixty,>\nprediction: <under the introus for may monee, nin the sixty,>\n\n'

## Testing

In [24]:
!pip install jiwer

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 61.7 MB/s eta 0:00:00:00:01


In [25]:
# import os
# model=None

# # Edit epoch number to start from saved epoch
# model = keras.models.load_model(
#     "my_model_v3.keras",
#     compile=False,
#     custom_objects={"Transformer": Transformer}
# )

# model.summary()

Model: "transformer_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ speech_feature_embedding_6      │ (None, None, 200)      │     1,164,400 │
│ (SpeechFeatureEmbedding)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_embedding_6               │ ?                      │        50,400 │
│ (TokenEmbedding)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_40 (Sequential)      │ ?                      │     3,095,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_decoder_6           │ ?                      │       804,600 │
│ (TransformerDecoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_76 (Dense)                │ ?                      │        10,452 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,961,052 (15.11 MB)

 Trainable params: 3,961,052 (15.11 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# val_data = data[split+5:]
# test_ds = create_tf_dataset(val_data, bs=4)

In [28]:
# from jiwer import wer,cer


# cnt=10
# targets=[]
# predictions=[]
# for i in ds:
#     cnt-=1
#     if cnt==0:
#         break
#     source=i['source']
#     target=i['target']
#     target_start_token_idx=2
#     target_end_token_idx=3
#     preds = model.generate(source, target_start_token_idx)
#     preds = preds.numpy()
    
#     target_text = "".join([idx_to_char[_] for _ in target[0, :]])
#     prediction = ""
#     for idx in preds[0, :]:
#         prediction += idx_to_char[idx]
#         if idx == target_end_token_idx:
#             break
#     print(f"target{cnt}:     {target_text.replace('-','')}")
#     print(f"prediction{cnt}: {prediction}\n")
#     targets.append(target_text.replace('-',''))
#     predictions.append(prediction)
# wer_score = wer(targets, predictions)*100
# cer_score = cer(targets, predictions)*100
# print(f"Word Error Rate: {wer_score:.4f}")
# print(f"Char Error Rate: {cer_score:.4f}")

    
# print("On test:")

# cnt=10

# targets=[]
# predictions=[]
# for i in test_ds:
#     cnt-=1
#     if cnt==0:
#         break
#     source=i['source']
#     target=i['target']
#     target_start_token_idx=2
#     target_end_token_idx=3
#     preds = model.generate(source, target_start_token_idx)
#     preds = preds.numpy()
    
#     target_text = "".join([idx_to_char[_] for _ in target[0, :]])
#     prediction = ""
#     for idx in preds[0, :]:
#         prediction += idx_to_char[idx]
#         if idx == target_end_token_idx:
#             break
#     print(f"target{cnt}:     {target_text.replace('-','')}")
#     print(f"prediction{cnt}: {prediction}\n")
#     targets.append(target_text.replace('-',''))
#     predictions.append(prediction)
# wer_score = wer(targets, predictions)*100
# cer_score = cer(targets, predictions)*100
# print(f"Word Error Rate: {wer_score:.4f}")
# print(f"Char Error Rate: {cer_score:.4f}")

target9:     <سنوات العمر هو متاخد يعني ب بيتعالجوا كان كان الأول سو سينسيتيف>
prediction9: <سنوات العمر هو متاخد يعني بيتعالجبي كان كان الأول سينس تاخد يعني بيتاعا البيها بيتد>

target8:     <الشرق للغرب الفرق إيه يا فندم الفرق إيه>
prediction8: <الشرق للفرق إيه في فندم الفرق إيه>

target7:     <بسبب اللعبه برضو سابهم ليه اللعبه هي أصلا اللعبه ديت>
prediction7: <بسبب اللعبه دي أصلا اللعبه يا أصلا اللعبه دي أصلا باللعبه دي>

target6:     <كما أشار سيادة اللواء إن هو تقدم الصفوف و هو كان أول واحد يستشهد يعني في هذه المعركه>
prediction6: <كما أشار سيادة اللواء إن هو تقدم الصفوف و هو كان أول واحد يستشهر سيادة اللواء إن هو كان أول واحد يستشهر سيادة اللواء إن هو>

target5:     <أسيب بنتي في سن صغير و أنزل أشتغل و هنا أنا بوجه التحيه فعلا لكل أم>
prediction5: <أسيب بنتي في سين صغير و أنا أنا بوجه اتحيه فعلا لكل أم من السن صغير و أنا أنا بوجه إحن>

target4:     <المنطقه فيها فهد>
prediction4: <المنصيتك فيها فهم>

target3:     <في داخلي سؤال يعني أستحي أن أسأله>
prediction3: <في داخلي سؤال يعن